<a href="https://colab.research.google.com/github/j143/in-car-voice-assistant/blob/colab/notebooks/edge_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!git stash save
!git pull https://github.com/j143/in-car-voice-assistant

Saved working directory and index state WIP on main: 653ad70 diable failing actions step
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 10 (delta 7), reused 10 (delta 7), pack-reused 0 (from 0)
Unpacking objects: 100% (10/10), 1.82 KiB | 372.00 KiB/s, done.
From https://github.com/j143/in-car-voice-assistant
 * branch            HEAD       -> FETCH_HEAD
Updating 653ad70..4257aa7
Fast-forward
 requirements-awq.txt   |  3 ++-
 scripts/awq_export.py  | 27 ++++++++++++++++---------
 scripts/export_awq.py  | 53 +++++++++++++++++++++++++++++++-------------------
 scripts/train_qlora.py |  1 -
 4 files changed, 53 insertions(+), 31 deletions(-)


In [4]:
%cd in-car-voice-assistant

/content/in-car-voice-assistant


In [3]:
%ls

checkpoints/       notebooks/                 requirements-train.txt
config/            pipeline/                  requirements.txt
data/              QUICKSTART_EXPERIMENTS.md  role.md
DEPLOYMENT.md      README.md                  scripts/
Dockerfile.edge    requirements-awq.txt       STRATEGIC_ALIGNMENT.md
Dockerfile.orin    requirements-min.txt       tests/
IMPLEMENTATION.md  requirements-nlu.txt
models/            requirements-rag.txt


In [6]:
import json
import subprocess
from pathlib import Path

## 1. Baseline Evaluation

Rule-based classifier on automotive test set.

In [7]:
import sys
import json
import subprocess

# Run baseline evaluation
result = subprocess.run(
    [sys.executable, "scripts/evaluate_domain.py", "--test", "data/automotive_domain_test.jsonl"],
    capture_output=True,
    text=True,
    env={"PYTHONPATH": "."},
)

print("--- Subprocess Standard Output ---")
print(result.stdout)
print("--- End Subprocess Standard Output ---")
print("--- Subprocess Standard Error ---")
print(result.stderr)
print("--- End Subprocess Standard Error ---")

# Attempt to parse the JSON output.
try:
    # Find the start and end of the JSON object in the stdout
    json_start_index = result.stdout.find('{')
    json_end_index = result.stdout.rfind('}')

    if json_start_index != -1 and json_end_index != -1 and json_end_index > json_start_index:
        json_string = result.stdout[json_start_index : json_end_index + 1]
        baseline = json.loads(json_string)
        print(json.dumps(baseline, indent=2))
    else:
        print("Warning: Could not find a complete JSON object in the subprocess output.")
        print("Please inspect the full output above.")

except json.JSONDecodeError as e:
    print(f"JSONDecodeError when parsing: {e}")
    print("Could not parse the extracted JSON string. Please inspect the full output above.")
except IndexError:
    print("Error: Subprocess stdout was empty or did not contain a newline before JSON parsing attempt.")


--- Subprocess Standard Output ---
{
  "total_samples": 15,
  "command_accuracy": 0.2,
  "technical_term_recognition_rate": 0.1667,
  "error_code_recognition_rate": 0.0,
  "avg_latency_ms": 57.88,
  "p95_latency_ms": 184.68,
  "latency_target_met": true,
  "accuracy_target_met": false
}

=== Performance Summary ===
Command Accuracy: 20.0% (Target: ≥90%)
Avg Latency: 57.88ms (Target: <100ms)
Technical Term Recognition: 16.7%
Error Code Recognition: 0.0%

✓ Accuracy Target Met: False
✓ Latency Target Met: True

--- End Subprocess Standard Output ---
--- Subprocess Standard Error ---
2025-12-16 06:56:28.699370: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765868188.720265    1206 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765868188.7261

## 2. QLoRA Fine-Tuning

Train LoRA adapters on automotive domain data.

In [10]:
!pip install -r requirements-train.txt

In [15]:
# Train QLoRA (requires GPU + requirements-train.txt)
!python scripts/train_qlora.py --model microsoft/phi-2 --epochs 3 --output checkpoints/automotive-adapter

2025-12-16 07:20:57.005088: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765869657.045556    7506 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765869657.055856    7506 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765869657.088925    7506 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765869657.088966    7506 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765869657.088975    7506 computation_placer.cc:177] computation placer alr

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base = "microsoft/phi-2"
adapter = "checkpoints/automotive-adapter"

tok = AutoTokenizer.from_pretrained(base, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    base,
    trust_remote_code=True,
    # device_map="cpu" removed as it's not supported for 4-bit/8-bit models
)
model = PeftModel.from_pretrained(model, adapter)
model.eval() # Set model to evaluation mode

print("Loaded with adapters:", model.peft_config.keys())
print("Trainable params:", sum(p.numel() for p in model.parameters() if p.requires_grad))

In [ ]:
# Example of testing the fine-tuned model with a sample query

# Prepare an input query based on the training data format
query = "User: My check engine light is on. What should I do?"
input_ids = tok(query, return_tensors="pt").input_ids.cuda()

# Generate a response from the fine-tuned model
with torch.no_grad():
    output_tokens = model.generate(input_ids, max_new_tokens=50)

# Decode the output and print it
decoded_output = tok.decode(output_tokens[0], skip_special_tokens=True)
print("--- Fine-tuned Model Response ---")
print(decoded_output)
print("---------------------------------")

print("\nThis output demonstrates a basic inference step with your fine-tuned model.")
print("To perform a more formal evaluation and compare it against the baseline,")
print("we would integrate this into an evaluation script (like the one used for the baseline). ")

## 3. AWQ Export

Export quantized model for edge deployment.

## 4. Comparison Table

| Configuration | Accuracy | Latency (ms) | Memory (GB) | Target Met |
|--------------|----------|--------------|-------------|------------|
| Baseline (rule) | 93.3% | 0.01 | <0.1 | ✓ Latency |
| + QLoRA | TBD | TBD | ~2.5 | TBD |
| + AWQ 4-bit | TBD | TBD | <2.0 | TBD |

In [1]:
%cd in-car-voice-assistant/

/content/in-car-voice-assistant


In [2]:
!python scripts/evaluate_intent.py data/automotive_domain_test.jsonl --classifier rule --rag off

Traceback (most recent call last):
  File "/content/in-car-voice-assistant/scripts/evaluate_intent.py", line 19, in <module>
    from pipeline.end_to_end import VoiceAssistantPipeline
ModuleNotFoundError: No module named 'pipeline'


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base = "microsoft/phi-2"
adapter = "checkpoints/automotive-adapter"

tok = AutoTokenizer.from_pretrained(base, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    base,
    trust_remote_code=True,
    # device_map="cpu" removed as it's not supported for 4-bit/8-bit models
)
model = PeftModel.from_pretrained(model, adapter)
model.eval() # Set model to evaluation mode

print("Loaded with adapters:", model.peft_config.keys())
print("Trainable params:", sum(p.numel() for p in model.parameters() if p.requires_grad))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
!pip install torch==2.9.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 749.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.4.0
    Uninstalling triton-3.4.0:
      Successfully uninstalled triton-3.4.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.27.3
    Uninstalling nvidia-nccl-cu12-2.27.3:
      Successfully uninstalled nvidia-nccl-cu12-2.27.3
  Attempting uninstall: torch
    Found existing installation: torch 2.8.0
    Uninstalling torch-2.8.0:
      Successfully uninstalled torch-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmcompressor 0.8.1 requires torch<=2.8.0,>=2.7.0, but you have torch 2.9.1 which is inco

## Archive

In [4]:
# Recompile bitsandbytes from source
!git clone https://github.com/TimDettmers/bitsandbytes.git
%cd bitsandbytes
!CUDA_VERSION=126 python setup.py install
%cd ../


Cloning into 'bitsandbytes'...
remote: Enumerating objects: 11598, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 11598 (delta 98), reused 64 (delta 57), pack-reused 11443 (from 3)
Receiving objects: 100% (11598/11598), 3.78 MiB | 9.84 MiB/s, done.
Resolving deltas: 100% (7885/7885), done.
/content/in-car-voice-assistant/bitsandbytes
/usr/local/lib/python3.12/dist-packages/setuptools/_distutils/dist.py:261: UserWarning: Unknown distribution option: 'cmake_source_dir'
  warnings.warn(msg)
configuration error: `project.license` must be valid exactly by one definition (2 matches found):

    - keys:
        'file': {type: string}
      required: ['file']
    - keys:
        'text': {type: string}
      required: ['text']

DESCRIPTION:
    `Project license <https://peps.python.org/pep-0621/#license>`_.

GIVEN VALUE:
    "MIT"

OFFENDING RULE: 'oneOf'

DEFINITION:
    {
        "oneOf": [
            {
                "pr

In [15]:
# Ensure peft is explicitly set to the compatible version in requirements-train.txt
!sed -i 's/^peft$/peft==0.7.1/g' requirements-train.txt
!cat requirements-train.txt

# Optional training/fine-tuning requirements
bitsandbytes
peft==0.7.1
transformers==4.38.0
torch
scikit-learn
pandas


In [16]:
# Reinstall all requirements, ensuring peft is downgraded to the compatible version
!pip install -r requirements-train.txt --force-reinstall --upgrade

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/bitsandbytes-0.49.1.dev0-py3.12-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached peft-0.7.1-py3-none-any.whl.metadata (25 kB)
  Using cached transformers-4.38.0-py3-none-any.whl.metadata (131 kB)
  Using cached torch-2.9.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached scikit_learn-1.8.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (11 kB)
  Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.3 MB/s eta 0:00:00
  Using cached tokenizer

In [1]:
# Example of testing the fine-tuned model with a sample query

# Prepare an input query based on the training data format
query = "User: My check engine light is on. What should I do?"
input_ids = tok(query, return_tensors="pt").input_ids.cuda()

# Generate a response from the fine-tuned model
with torch.no_grad():
    output_tokens = model.generate(input_ids, max_new_tokens=50)

# Decode the output and print it
decoded_output = tok.decode(output_tokens[0], skip_special_tokens=True)
print("--- Fine-tuned Model Response ---")
print(decoded_output)
print("---------------------------------")

print("\nThis output demonstrates a basic inference step with your fine-tuned model.")
print("To perform a more formal evaluation and compare it against the baseline,")
print("we would integrate this into an evaluation script (like the one used for the baseline).")

[Errno 2] No such file or directory: 'in-car-voice-assistant/'
/content


In [3]:
# Train QLoRA (requires GPU + requirements-train.txt)
!python scripts/train_qlora.py --model microsoft/phi-2 --epochs 3 --output checkpoints/automotive-adapter

2025-12-16 04:36:43.631197: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765859803.652950   31160 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765859803.659935   31160 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765859803.677158   31160 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765859803.677182   31160 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765859803.677186   31160 computation_placer.cc:177] computation placer alr

In [25]:
!pip install -r requirements-awq.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 130.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 826.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 798.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 139.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
# Reinstall remaining requirements-awq.txt dependencies (transformers, accelerate), which should keep torch==2.9.1
!pip install --force-reinstall -r requirements-awq.txt

  Using cached llmcompressor-0.8.1-py3-none-any.whl.metadata (11 kB)
  Using cached loguru-0.7.3-py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.3 MB/s eta 0:00:00
  Using cached torch-2.8.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached transformers-4.56.2-py3-none-any.whl.metadata (40 kB)
  Using cached accelerate-1.10.1-py3-none-any.whl.metadata (19 kB)
  Using cached nvidia_ml_py-13.580.82-py3-none-any.whl.metadata (9.6 kB)
  Using cached compressed_tensors-0.12.2-py3-none-any.whl.metadata (7.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manyl

In [4]:
# Now install llmcompressor hoping it adapts to the existing torch==2.9.1, or gives a clear conflict
!pip install llmcompressor[transformers]

In [10]:
!pip install llmcompressor

  Using cached llmcompressor-0.8.1-py3-none-any.whl.metadata (11 kB)
Using cached llmcompressor-0.8.1-py3-none-any.whl (273 kB)


In [11]:
# Export AWQ (requires requirements-awq.txt)
!python scripts/export_awq.py --model microsoft/phi-2 --adapter checkpoints/automotive-adapter --output exports/automotive-awq-4bit

AWQ dependencies not available. Install requirements-awq.txt
pip install 'llm-compressor[transformers]'


In [6]:
from llmcompressor.transformers import oneshot
from llmcompressor.modifiers.quantization import GPTQModifier
from transformers import AutoTokenizer, AutoModelForCausalLM

ModuleNotFoundError: Could not import module 'PreTrainedModel'. Are this object's requirements defined correctly?